In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [2]:
np

<module 'numpy' from '/opt/conda/lib/python3.7/site-packages/numpy/__init__.py'>

In [3]:
# Reading input

# articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
# customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [4]:
transactions_train['max_purc_date'] = transactions_train.groupby('customer_id')['t_dat'].transform('max')

transactions_train['min_purc_date'] = transactions_train.groupby('customer_id')['t_dat'].transform('min')

transactions_train['total_visits'] = transactions_train.groupby('customer_id')['t_dat'].transform('nunique')

transactions_train['total_item_bought'] = transactions_train.groupby('customer_id')['article_id'].transform('nunique')

transactions_train['max_year'] = transactions_train['max_purc_date'].str.slice(0,4).astype(int)

In [5]:
transactions_train_filt = transactions_train[transactions_train['max_year'] == 2020]

# transactions_train_filt['product_reach'] = transactions_train_filt.groupby('article_id')['customer_id'].transform('nunique')

# transactions_train_filt = transactions_train_filt[transactions_train_filt['product_reach'] > 8]

transactions_train_filt['repeat_purchase'] = transactions_train_filt.groupby(['customer_id','article_id'])['t_dat'].transform('nunique')

transactions_train_filt = transactions_train_filt[transactions_train_filt['repeat_purchase'] > 1]

# transactions_train_filt = transactions_train_filt[transactions_train_filt['total_visits'] >= 2]

transactions_train_filt = transactions_train_filt[transactions_train_filt['total_item_bought'] >= 12]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
transactions_train_filt.shape

(2485576, 11)

In [7]:
transactions_train_filt['t_dat_datetime']  = pd.to_datetime(transactions_train_filt['t_dat'])

In [8]:
product_purchase_cycle = transactions_train_filt[['customer_id','article_id','t_dat_datetime']].\
sort_values(['customer_id','article_id','t_dat_datetime']).drop_duplicates()

In [9]:
product_purchase_cycle['t_dat_datetime_lag'] = product_purchase_cycle.\
groupby(['customer_id','article_id'])['t_dat_datetime'].shift(1)

In [10]:
product_purchase_cycle['purchase_cycle'] = product_purchase_cycle['t_dat_datetime'] - product_purchase_cycle['t_dat_datetime_lag']

In [11]:
Average_Purchase_Cycle = product_purchase_cycle[~(product_purchase_cycle['purchase_cycle'].isnull())].\
groupby(['customer_id','article_id']).agg({"purchase_cycle":"median"}).reset_index()

In [12]:
transactions_train_filt = pd.merge(transactions_train_filt,
                                   Average_Purchase_Cycle,
                                   left_on = ['customer_id','article_id'],
                                   right_on = ['customer_id','article_id'],
                                   how='inner'
                                  )

In [13]:
transactions_train_filt.shape

(2485576, 13)

In [14]:
transactions_train_filt['product_last_purchase_date'] = transactions_train_filt.\
groupby(['customer_id','article_id'])['t_dat_datetime'].transform("max")

In [15]:
from datetime import datetime
from datetime import timedelta

In [16]:
transactions_train_filt['purchase_cycle_days'] = transactions_train_filt['purchase_cycle'].astype('str').str.split(' ', expand=True)[0]

In [17]:
transactions_train_filt['next_purchase_date'] = transactions_train_filt['product_last_purchase_date'] + transactions_train_filt['purchase_cycle_days'].astype(int).map(timedelta)

In [18]:
start_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(-90)
end_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(90)

In [19]:
datetime.date(transactions_train_filt['t_dat_datetime'].max()) 

datetime.date(2020, 9, 22)

In [20]:
start_date, end_date

(Timestamp('2020-06-24 00:00:00'), Timestamp('2020-12-21 00:00:00'))

In [21]:
len(transactions_train_filt['customer_id'].unique())

285320

In [22]:
customer_next_purchase = transactions_train_filt[transactions_train_filt['next_purchase_date'].\
                        between(start_date, end_date, inclusive="both")]\
[['customer_id','t_dat','article_id']].drop_duplicates()

In [23]:
customer_next_purchase.shape

(404090, 3)

In [24]:
customer_next_purchase['repeat_purchase'] = customer_next_purchase.groupby(['customer_id'])['article_id'].transform('count')

In [25]:
customer_next_purchase['product_imp_rank'] = customer_next_purchase.groupby(['customer_id'])['repeat_purchase'].rank(method="first",ascending=True)

In [26]:
customer_next_purchase['article_id'] = ' 0'+customer_next_purchase['article_id'].astype('str')

In [27]:
top_12_products = pd.DataFrame(customer_next_purchase[customer_next_purchase['product_imp_rank']<=12]\
                               [['customer_id','article_id']].drop_duplicates().\
                               groupby(['customer_id']).article_id.sum().reset_index())

In [28]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')\
[['customer_id']]

In [29]:
sample_submission = pd.merge(sample_submission,
                             customer_next_purchase[customer_next_purchase['product_imp_rank']<=12]\
                             [['customer_id','article_id']].\
                             drop_duplicates(),
                             left_on = ['customer_id'],
                             right_on = ['customer_id'],
                             how = 'left'
                            )

In [30]:
sample_submission.shape

(1443140, 2)

In [50]:
no_products_cust = sample_submission[sample_submission['article_id'].isna()]
atleast_1_products_cust = sample_submission.dropna(axis=0)

no_products_cust.shape, atleast_1_products_cust.shape

In [ ]:
# top_12_products = pd.DataFrame(atleast_1_products_cust.groupby(['customer_id']).article_id.sum().reset_index())

In [46]:
top_12_repeat_purchase_products = transactions_train_filt.groupby('article_id').agg({'customer_id':"nunique"}).\
sort_values('customer_id', ascending=False).reset_index().head(12)[['article_id']]

top_12_repeat_purchase_products['article_id'] = ' 0'+top_12_repeat_purchase_products['article_id'].astype('str')

In [51]:
# no_products_cust['article_id'].fillna( "0706016001 0706016002 0372860001 0399223001 0464297007 0160442007 0610776002 0156231001 0399256001 0372860002 0706016003 0759871002", inplace=True)
# no_products_cust.shape
no_products_cust['article_id'].fillna( top_12_repeat_purchase_products.article_id.sum(), inplace=True)
no_products_cust.shape

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


(1267832, 2)

In [ ]:
atleast_1_products_cust_full = pd.DataFrame({'customer_id':[],'article_id':[]})
atleast_1_products_cust_full

for customer_id in atleast_1_products_cust['customer_id'].unique().tolist():
    customer_id_list = []
    article_id_list = []
    cust_products = atleast_1_products_cust[atleast_1_products_cust['customer_id'] == customer_id]\
    ['article_id'].unique().tolist()
    
    if len(cust_products) < 12:
        product_to_add = list(set(top_12_repeat_purchase_products['article_id'].unique().tolist()) -\
                              set (cust_products))
        
        for article_id in product_to_add[0: (12 - len(cust_products))]:
            customer_id_list.append(customer_id)
            article_id_list.append(article_id)
            
        extra_products = pd.DataFrame({"customer_id":customer_id_list,'article_id':article_id_list})
            
        atleast_1_products_cust_full = pd.concat([atleast_1_products_cust_full, extra_products], axis = 0)
    
    atleast_1_products_cust_full = pd.concat([atleast_1_products_cust_full, 
                                             atleast_1_products_cust[atleast_1_products_cust['customer_id'] == customer_id]]
                                             , axis = 0)

In [ ]:
final_submisison.to_csv('submission.csv', index =False)

In [ ]:
sample_df = atleast_1_products_cust[:12]

In [ ]:
cust_list = sample_df['customer_id'].unique().tolist()

In [ ]:
for i in cust_list:
    [sample_df['customer_id'] == i]['article_id'].unique().tolist()
    

In [ ]:
(sample_df[sample_df['customer_id'] == '00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75']\
['article_id'].unique().tolist()[0].strip())

In [ ]:
product_list = ["0706016001", "0706016002",
                "0372860001", '0399223001', '0464297007', '0160442007', 
                '0610776002', '0156231001', '0399256001', '0372860002',
                '0706016003', '0759871002']

In [ ]:
for i in product_list